<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Text Classification with Universal Sentence Encoder-localhost</H1></u></center>

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
messages = ["That band rocks!", "That song is really cool."]

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

INFO:tensorflow:Using C:\Users\map25\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 50.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 100.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 140.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 180.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 220.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 260.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 300.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder-large/3: 330.00MB
INFO:tensorflow:Download

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('../data/movie_reviews.csv.bz2', compression='bz2')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [6]:
data['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in data['sentiment'].values]
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
reviews = data['review'].values
sentiments = data['sentiment'].values

train_reviews = reviews[:30000]
train_sentiments = sentiments[:30000]

val_reviews = reviews[30000:35000]
val_sentiments = sentiments[30000:35000]

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
train_reviews.shape, val_reviews.shape, test_reviews.shape

((30000,), (5000,), (15000,))

In [8]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

In [9]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [10]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [11]:
def expand_contractions(text):
    return contractions.fix(text)

In [12]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

In [14]:
def pre_process_document(document):
    
    # strip HTML
    document = strip_html_tags(document)
    
    # lower case
    document = document.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    document = remove_accented_chars(document)
    
    # expand contractions    
    document = expand_contractions(document)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
        
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document

pre_process_corpus = np.vectorize(pre_process_document)

In [15]:
train_reviews = pre_process_corpus(train_reviews)
val_reviews = pre_process_corpus(val_reviews)
test_reviews = pre_process_corpus(test_reviews)

In [16]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, 
    batch_size=256, num_epochs=None, shuffle=True)

In [17]:
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, shuffle=False)

In [18]:
# Prediction on the whole validation set.
predict_val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': val_reviews}, val_sentiments, shuffle=False)

In [19]:
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': test_reviews}, test_sentiments, shuffle=False)

In [20]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 48.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 68.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 108.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 128.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 178.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 188.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 238.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 278.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 318.00MB
INFO:tensorflow:Downloading https://tfhub.dev/google/universal-sentence-enc

In [21]:
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\map25\\AppData\\Local\\Temp\\tmpiuajx4tq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000014A933F0D68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
import time

TOTAL_STEPS = 500
STEP_SIZE = 100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt.
INFO:tensorflow:loss = 178.04242, step = 0
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\map25\Ap

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-12-04:03:22
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.8448, accuracy_baseline = 0.505, auc = 0.9277709, auc_precision_recall = 0.9238656, average_loss = 0.34711793, global_step = 200, label/mean = 0.495, loss = 43.38974, precision = 0.819481, prediction/mean = 0.52800876, recall = 0.88040406
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 200: C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt-200
Eval Metrics (Validation): {'accuracy': 0.8448, 'accuracy_baseline': 0.505, 'auc': 0.9277709, 'auc_precision_recall': 0.9238656, 'average_loss': 0.34711793, 'label/mean': 0.495, 'loss': 43.38974, 'precision': 0.819481, 'prediction/mean': 0.52800876, 'recall': 0.88040406, 'global_step': 200}

----------------------------------------------------------------------------------------------------
Training for step = 200
INFO:ten

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-12-04:09:44
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.849, accuracy_baseline = 0.505, auc = 0.9301559, auc_precision_recall = 0.9257468, average_loss = 0.34161478, global_step = 400, label/mean = 0.495, loss = 42.70185, precision = 0.82428354, prediction/mean = 0.52780706, recall = 0.8832323
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 400: C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt-400
Eval Metrics (Validation): {'accuracy': 0.849, 'accuracy_baseline': 0.505, 'auc': 0.9301559, 'auc_precision_recall': 0.9257468, 'average_loss': 0.34161478, 'label/mean': 0.495, 'loss': 42.70185, 'precision': 0.82428354, 'prediction/mean': 0.52780706, 'recall': 0.8832323, 'global_step': 400}

----------------------------------------------------------------------------------------------------
Training for step = 400
INFO:ten

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-12-04:16:45
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.8564, accuracy_baseline = 0.505, auc = 0.9321009, auc_precision_recall = 0.9275272, average_loss = 0.3323834, global_step = 600, label/mean = 0.495, loss = 41.547924, precision = 0.85323685, prediction/mean = 0.49773434, recall = 0.8573737
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 600: C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt-600
Eval Metrics (Validation): {'accuracy': 0.8564, 'accuracy_baseline': 0.505, 'auc': 0.9321009, 'auc_precision_recall': 0.9275272, 'average_loss': 0.3323834, 'label/mean': 0.495, 'loss': 41.547924, 'precision': 0.85323685, 'prediction/mean': 0.49773434, 'recall': 0.8573737, 'global_step': 600}


In [23]:
dnn.evaluate(input_fn=predict_train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-12-04:16:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-12-04:18:05
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.8633, accuracy_baseline = 0.5005, auc = 0.938951, auc_precision_recall = 0.93868655, average_loss = 0.31627437, global_step = 600, label/mean = 0.5005, loss = 40.375454, precision = 0.87051874, prediction/mean = 0.4934998, recall = 0.85387945
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 600: C:\Users\map25\AppData\Local\Temp

{'accuracy': 0.8633,
 'accuracy_baseline': 0.5005,
 'auc': 0.938951,
 'auc_precision_recall': 0.93868655,
 'average_loss': 0.31627437,
 'label/mean': 0.5005,
 'loss': 40.375454,
 'precision': 0.87051874,
 'prediction/mean': 0.4934998,
 'recall': 0.85387945,
 'global_step': 600}

In [24]:
dnn.evaluate(input_fn=predict_test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-12-04:18:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\map25\AppData\Local\Temp\tmpiuajx4tq\model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-12-04:19:03
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.85973334, accuracy_baseline = 0.5006667, auc = 0.93549514, auc_precision_recall = 0.93464446, average_loss = 0.32488343, global_step = 600, label/mean = 0.5006667, loss = 41.29874, precision = 0.8667571, prediction/mean = 0.4959114, recall = 0.8505992
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 600: C:\Users\map25\AppData\L

{'accuracy': 0.85973334,
 'accuracy_baseline': 0.5006667,
 'auc': 0.93549514,
 'auc_precision_recall': 0.93464446,
 'average_loss': 0.32488343,
 'label/mean': 0.5006667,
 'loss': 41.29874,
 'precision': 0.8667571,
 'prediction/mean': 0.4959114,
 'recall': 0.8505992,
 'global_step': 600}